https://towardsdatascience.com/linear-programming-and-discrete-optimization-with-python-using-pulp-449f3c5f6e99

Git repo
https://github.com/tirthajyoti/Optimization-Python

You are in charge of the diet plan for high school lunch. Your job is to make sure that the students get the right balance of nutrition from the chosen food.

The discrete optimization problem is simple: Minimize the cost of the lunch given these constraints (on total calories but also on each of the nutritional component e.g. protein etc.)

${minimize} \sum C_i\dot f_i$

Constraints

${carbs_{low}} < \sum Carb_i \dot f_i < carbs_{hi}$

${fat_{low}} < \sum Fat_i \dot f_i < fat_{hi}$

...

Notice that the inequality relations are all linear in nature i.e. the variables f are multiplied by constant coefficients and the resulting terms are bounded by constant limits and that’s what makes this problem solvable by an LP technique.

In [56]:
import pulp 
from pulp import *
import pandas as pd

Create a linear programming minimization problem

In [67]:

prob = LpProblem("Diet Problem",LpMinimize)


In [58]:
# Read the first few rows of dataset in a Pandas DataFrame
# Reads only the nutrition info not the bounds/constraints
df = pd.read_excel("data/diet - medium.xls",nrows=17)

Use dictionaries to store tabulated information for each nutrition componet of interest

In [59]:
# Create a list of the food items
food_items = list(df['Foods'])

# Create a dictinary of costs for all food items
costs = dict(zip(food_items,df['Price/Serving']))

# Create a dictionary of calories for all food items
calories = dict(zip(food_items,df['Calories']))

# Create a dictionary of total fat for all food items
fat = dict(zip(food_items,df['Total_Fat (g)']))

# Create a dictionary of carbohydrates for all food items
carbs = dict(zip(food_items,df['Carbohydrates (g)']))

# Create a dictionary of carbohydrates for all food items
fiber = dict(zip(food_items,df['Dietary_Fiber (g)']))

# Create a dictionary of carbohydrates for all food items
protein = dict(zip(food_items,df['Protein (g)']))

Create a dictionary of food items variables with:
- lower bound (e.g. 0) on the quantity assigned to each food
- category `continuous` i.e. the optimization solution can take any real-numbered value greater than zero.

In [60]:
food_vars = LpVariable.dicts("Food",           # name
                             food_items,       # list of items we created earlier
                             lowBound=0,       # lower bound
                             cat='Continuous'  # solution data type
                            )

Main objective function (i.e. function to minimize)

In [61]:
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

Constraints

In [62]:
# Calories
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0,  "CalsMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0, "CalsMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0, "CarbsMaximum"

# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0, "FatMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 60.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 125.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 100.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 150.0, "ProteinMaximum"



Find solution

In [63]:
prob.solve()

print("Status:", LpStatus[prob.status])

Status: Optimal


Display solution

In [64]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Food_Frozen_Broccoli = 6.9242113
Food_Scrambled_Eggs = 6.060891
Food__Baked_Potatoes = 1.0806324


Objective function, total cost of meal. 

Optimized value is minimumm cost given nutritional constraints

In [66]:
obj = value(prob.objective)
print(f"The total cost of this balanced meal is: £ {round(obj,2)}")

The total cost of this balanced meal is: £ 5.52
